In [ ]:
import requests
import json

api_key = "eBUlsPb5gbBKzGmRb6j9jgsWU7pePqvmdK19ngIchBV9VsNhyM6h5rUHSCCQgpOQ4hxXcK1ZU3zlYJlk7j2e18F3tsVYqUmJbimoacx5jDSXOpx-4tvszGavVFgoZXYx"

cuisine_types = ["chinese", "italian", "mexican", "indian", "japanese", "french", "korean"]
unique_restaurant_ids = set()
restaurant_data = []
for cuisine in cuisine_types:
    # Define the search term with the cuisine type
    search_term = f"{cuisine} restaurants"

    # Initialize variables for pagination
    offset = 0
    limit = 50

    # Continue making requests until you reach your desired number of results
    desired_total_results = 1000

    while offset < desired_total_results:
        # Make a request to the Yelp API
        url = "https://api.yelp.com/v3/businesses/search"
        headers = {
            "Authorization": f"Bearer {api_key}"
        }
        params = {
            "location": "Manhattan",
            "term": search_term,
            "limit": limit,
            "offset": offset
        }

        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        restaurants = data["businesses"]
        for restaurant in restaurants:
          restaurant_id = restaurant["id"]
          restaurant['cuisine'] = cuisine

          # Check if the restaurant ID is already in the set
          if restaurant_id not in unique_restaurant_ids:
              # Add the restaurant ID to the set to mark it as seen
              unique_restaurant_ids.add(restaurant_id)
              restaurant_data.append(restaurant)

        # Increment the offset for the next page of results
        offset += limit


In [ ]:
import csv
from datetime import datetime

# Define the CSV file path
csv_file_path = "restaurant_data_cuisines.csv"

# Write the collected data to a CSV file
with open(csv_file_path, mode='w', newline='') as csv_file:
    fieldnames = ['Business ID', 'Name', 'Address', 'Coordinates', 'Number of Reviews', 'Rating', 'Zip Code', 'insertedAtTimestamp', 'Cuisine']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Write each restaurant's data as a row in the CSV file
    for restaurant in restaurant_data:

        writer.writerow({
            'Business ID': restaurant.get('id', ''),
            'Name': restaurant.get('name', ''),
            'Address': restaurant.get('location', {}).get('address1', ''),
            'Coordinates': f"{restaurant.get('coordinates', {}).get('latitude', '')}, {restaurant.get('coordinates', {}).get('longitude', '')}",
            'Number of Reviews': restaurant.get('review_count', ''),
            'Rating': restaurant.get('rating', ''),
            'Zip Code': restaurant.get('location', {}).get('zip_code', ''),
            'insertedAtTimestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'Cuisine': restaurant.get('cuisine', '')
        })

print(f'Results stored in {csv_file_path}')

Results stored in restaurant_data_cuisines.csv
